In [1]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

# Power consumption and Dark Silicon

## How fast are my CPUs and GPUs?

In [2]:
! lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   24
  On-line CPU(s) list:    0-23
Vendor ID:                GenuineIntel
  Model name:             13th Gen Intel(R) Core(TM) i7-13700
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   16
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   57%
    CPU max MHz:          5200.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4224.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc c

In [3]:
! cat /proc/cpuinfo |grep MHz

cpu MHz		: 1805.923
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 3027.436
cpu MHz		: 800.000
cpu MHz		: 3040.523
cpu MHz		: 800.000
cpu MHz		: 3075.390
cpu MHz		: 800.000
cpu MHz		: 2983.953
cpu MHz		: 800.000
cpu MHz		: 2197.679
cpu MHz		: 2116.265
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 800.000
cpu MHz		: 2197.785


In [4]:
! ssh htseng@eevee "lscpu"

Architecture:                       x86_64
CPU op-mode(s):                     32-bit, 64-bit
Byte Order:                         Little Endian
Address sizes:                      46 bits physical, 48 bits virtual
CPU(s):                             32
On-line CPU(s) list:                0-31
Thread(s) per core:                 2
Core(s) per socket:                 8
Socket(s):                          2
NUMA node(s):                       2
Vendor ID:                          GenuineIntel
CPU family:                         6
Model:                              85
Model name:                         Intel(R) Xeon(R) Silver 4108 CPU @ 1.80GHz
Stepping:                           4
CPU MHz:                            800.576
CPU max MHz:                        3000.0000
CPU min MHz:                        800.0000
BogoMIPS:                           3600.00
Virtualization:                     VT-x
L1d cache:                          512 KiB
L1i cache:                          512 KiB
L2 

In [29]:
# Operating at lowest freq.
! ssh htseng@eevee "/nfshome/htseng/courses/CSE142/demo/power/lowerMaxFreq.sh; cd /nfshome/htseng/courses/CSE142/demo/power; make -C matrix_mul clean all ;  time sudo likwid-perfctr -g ENERGY matrix_mul/blockmm 1024 16" | grep "STAT"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  | 49531287594 |   0 | 49523240493 | 1.547853e+09 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  | 17833856848 |   0 | 17820370015 | 5.573080e+08 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  | 40126212576 |   0 | 40095866160 |   1253944143 |
|       TEMP_CORE STAT       |   TMP0  |         711 |  17 |          27 |      22.2188 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |    862.7824 |   0 |    446.5026 |      26.9620 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |           0 |   0 |           0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |     70.0977 |   0 |     59.8332 |       2.1906 |
|  Runtime (RDTSC) [s] STAT |   716.0672 |  22.3771 |  22.3771 |  22.3771 |
| Runtime unhalted [s] STAT |     9.9078 |        0 |   9.9002 |   0.3096 |
|      Clock [MHz] STAT     | 12795.5533 | 796.3893 | 800.8916 | 399.8610 |
| 

In [30]:
# Allowing turbo boost
! ssh htseng@eevee "/nfshome/htseng/courses/CSE142/demo/power/restoreMaxFreq.sh; cd /nfshome/htseng/courses/CSE142/demo/power; time sudo likwid-perfctr -g ENERGY matrix_mul/blockmm 1024 16" | grep "STAT"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  | 49525949879 |   0 | 49523236402 | 1.547686e+09 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  | 17850389183 |   0 | 17846095703 | 5.578247e+08 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  | 10736840448 |   0 | 10727345304 |    335526264 |
|       TEMP_CORE STAT       |   TMP0  |         743 |  18 |          27 |      23.2188 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |    320.2301 |   0 |    174.1351 |      10.0072 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |           0 |   0 |           0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |     39.5498 |   0 |     22.9646 |       1.2359 |
|  Runtime (RDTSC) [s] STAT |   191.8624 |   5.9957 |    5.9957 |   5.9957 |
| Runtime unhalted [s] STAT |     9.9169 |        0 |    9.9145 |   0.3099 |
|      Clock [MHz] STAT     | 14947.4945 | 798.8559 | 2994.4930 | 467.1092 |

In [8]:
# Operating at lowest freq.
! ssh htseng@eevee "/nfshome/htseng/courses/CSE142/demo/power/lowerMaxFreq.sh; cd /nfshome/htseng/courses/CSE142/demo/power/popcounts; make clean; make; time sudo likwid-perfctr -g ENERGY ./popcount_D" | grep "STAT"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  | 80118744646 |   0 | 80002272949 | 2.503711e+09 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  | 21473661358 |   0 | 21389051661 | 6.710519e+08 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  | 48315649176 |   0 | 48125278584 | 1.509864e+09 |
|       TEMP_CORE STAT       |   TMP0  |         730 |  18 |          27 |      22.8125 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |   1032.6894 |   0 |    534.1400 |      32.2715 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |           0 |   0 |           0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |     79.5384 |   0 |     72.1068 |       2.4856 |
|  Runtime (RDTSC) [s] STAT |   857.4368 |  26.7949 |  26.7949 |  26.7949 |
| Runtime unhalted [s] STAT |    11.9299 |        0 |  11.8829 |   0.3728 |
|      Clock [MHz] STAT     | 16000.1414 | 798.4320 | 801.3755 | 500.0044 |
| 

In [9]:
# Allowing turbo boost
! ssh htseng@eevee "/nfshome/htseng/courses/CSE142/demo/power/restoreMaxFreq.sh; cd /nfshome/htseng/courses/CSE142/demo/power/popcounts;  sudo likwid-perfctr -g ENERGY ./popcount_D" | grep "STAT"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  | 80005178427 |   0 | 80002268036 | 2.500162e+09 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  | 21387905485 |   0 | 21383241417 | 6.683720e+08 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  | 12880339128 |   0 | 12869923608 | 4.025106e+08 |
|       TEMP_CORE STAT       |   TMP0  |         765 |  19 |          28 |      23.9062 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |    382.4405 |   0 |    207.3791 |      11.9513 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |           0 |   0 |           0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |     46.4689 |   0 |     27.6189 |       1.4522 |
|  Runtime (RDTSC) [s] STAT |   229.1744 |   7.1617 |    7.1617 |   7.1617 |
| Runtime unhalted [s] STAT |    11.8821 |        0 |   11.8796 |   0.3713 |
|      Clock [MHz] STAT     | 13767.5719 | 799.6714 | 2990.6788 | 430.2366 |

In [37]:
# Operating at lowest freq.
! ssh htseng@eevee "/nfshome/htseng/courses/CSE142/demo/power/lowerMaxFreq.sh; cd /nfshome/htseng/courses/CSE142/demo/power;  time sudo likwid-perfctr -g ENERGY matrix_mul/blockmm_pthread 2048 16 32" | grep "STAT"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  |  74620663636 |   0 |  74471251336 | 2.331896e+09 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  |  51860027547 |   0 |  51732339323 | 1.620626e+09 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  | 116685053424 |   0 | 116397755784 | 3.646408e+09 |
|       TEMP_CORE STAT       |   TMP0  |          720 |  18 |           27 |      22.5000 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |    2508.4845 |   0 |    1300.1085 |      78.3901 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |            0 |   0 |            0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |     202.3557 |   0 |     170.1228 |       6.3236 |
|  Runtime (RDTSC) [s] STAT |  2081.8208 |  65.0569 |   65.0569 |  65.0569 |
| Runtime unhalted [s] STAT |    28.8111 |        0 |   28.7403 |   0.9003 |
|      Clock [MHz] STAT     | 24000.8025 | 799.2857 |  800.740

In [36]:
# Allowing turbo boost
! ssh htseng@eevee "/nfshome/htseng/courses/CSE142/demo/power/restoreMaxFreq.sh; cd /nfshome/htseng/courses/CSE142/demo/power;  time sudo likwid-perfctr -g ENERGY matrix_mul/blockmm_pthread 2048 16 4" | grep "STAT"

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  | 74479237353 |   0 | 65212611945 | 2.327476e+09 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  | 52458964386 |   0 | 45941907601 | 1.639343e+09 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  | 31508457624 |   0 | 27570327480 | 9.846393e+08 |
|       TEMP_CORE STAT       |   TMP0  |         759 |  19 |          28 |      23.7188 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |    966.6942 |   0 |    538.4854 |      30.2092 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |           0 |   0 |           0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |    113.2519 |   0 |     67.4621 |       3.5391 |
|  Runtime (RDTSC) [s] STAT |   563.4432 |  17.6076 |   17.6076 |  17.6076 |
| Runtime unhalted [s] STAT |    29.1446 |        0 |   25.5239 |   0.9108 |
|      Clock [MHz] STAT     | 16422.2851 | 799.9077 | 2999.3609 | 513.1964 |